# <div align="center"><b> PINECONE EXAMPLE </b></div>

<div align="right">📝 <em><small><font color='Gray'>Nota:</font></small></em></div>

<div align="right"> <em><small><font color='Gray'> La funcionalidad de visualización de jupyter notebooks en <a href="https://github.com/" target="_blank">github</a> es solamente un preview.</font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Para mejor visualización se sugiere utilizar el visualizador recomendado por la comunidad: <a href="https://nbviewer.org/" target="_blank">nbviewer</a></font></small></em> </div>

<div align="right"> <em><small><font color='Gray'> Puedes a acceder al siguiente enlace para ver este notebook en dicha página: <a href="https://nbviewer.org/github/brunomaso1/uba-mia/blob/mia-nlp2/mia-nlp2/Ejemplos/pinecone-example.ipynb">Pinecone example</a></font></small></em> </div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

<!-- Colab -->
<!-- <div align="center"><img src="https://drive.google.com/uc?export=view&id=1QSNrTsz1hQbmZwpgwx0qpfpNtLW19Orm" width="600" alt="Figura 1: A data scientist is working on word generation using the Lord of the Rings lore. The image is dark and moody, with a focus on the scientist's computer screen. The screen displays a visualization the one ring, with a map of Middle Earth in the background. - Generada con DALL-E3"></div> -->

<div align="center"><img src="../resources/img1.jpeg" width="600" alt="Figura 1: A data scientist is sitting in front of a computer screen, intently focused on the task at hand. The room is dimly lit, with the only light coming from the computer screen. The neural network is displayed on the screen, with the data scientist working to. The nerual network is deepseek R1. - Generada con Microsoft Image Creator"></div>

<div align="center"><small><em>Figura 1: A data scientist is sitting in front of a computer screen, intently focused on the task at hand. The room is dimly lit, with the only light coming from the computer screen. The neural network is displayed on the screen, with the data scientist working to. The nerual network is deepseek R1. - Generada con Microsoft Image Creator</em></small></div>

<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Ejemplo de prueba de Pinecone                                                                                                              |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | Ejemplo de prueba de funcionamiento utilización de Pinecone como base de datos vectorial                                                             |
| **Integrantes** | Bruno Masoller (brunomaso1@gmail.com)                                                                                                  |

</div>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻

In [ ]:
%pip install python-dotenv # Para cargar variables de entorno desde un archivo .env
%pip install pinecone # Para la integración con Pinecone
%pip install sentence-transformers # Para la generación de embeddings
%pip install huggingface_hub[hf_xet] # Para la carga de modelos de Hugging Face

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [48]:
import os, time
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [49]:
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
EMBEDDINGS_DIMENSION = 384 # Para el modelo sentence-transformers/all-MiniLM-L6-v2
EMBEDDINGS_MODEL = "all-MiniLM-L6-v2"

## Consinga

Este notebook es un ejemplo de prueba de funcionamiento de la utilización de Pinecone como base de datos vectorial. En este caso, se utiliza para almacenar y recuperar embeddings generados por un modelo de lenguaje. El objetivo es demostrar cómo se pueden utilizar los embeddings para realizar búsquedas semánticas y encontrar información relevante en un conjunto de datos.

## Resolución

La documentación de Pinecone es muy completa y se puede encontrar en su [página oficial](https://docs.pinecone.io/docs/quickstart). En este notebook, se utilizará la API de Pinecone para crear un índice, insertar datos y realizar consultas. Se utilizará el modelo de lenguaje `multilingual-e5-large` de OpenAI para generar los embeddings.

In [50]:
pc = Pinecone(api_key=PINECONE_API_KEY)

### Creación de un índice

Ejemplo de creación de un índice en Pinecone desde un modelo:

In [51]:
# https://docs.pinecone.io/guides/indexes/create-an-index#integrated-embedding
index_name_from_model = "example-index-from-model"

if not pc.has_index(index_name_from_model):
    pc.create_index_for_model(
        name=index_name_from_model,
        cloud="aws",
        region="us-east-1",
        embed={"model": "multilingual-e5-large", "field_map": {"text": "chunk_text"}},
    )

In [52]:
pc.describe_index(index_name_from_model)

{
    "name": "example-index-from-model",
    "metric": "cosine",
    "host": "example-index-from-model-22dypz1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null,
    "embed": {
        "model": "multilingual-e5-large",
        "field_map": {
            "text": "chunk_text"
        },
        "dimension": 1024,
        "metric": "cosine",
        "write_parameters": {
            "input_type": "passage",
            "truncate": "END"
        },
        "read_parameters": {
            "input_type": "query",
            "truncate": "END"
        },
        "vector_type": "dense"
    }
}

Ejemplo de creación de un indice con embeddings ya generados:

In [53]:
# https://docs.pinecone.io/guides/indexes/create-an-index#bring-your-own-vectors
index_name_from_embeddings = "example-index-from-embeddings"

if not pc.has_index(index_name_from_embeddings):
    pc.create_index(
        name=index_name_from_embeddings,
        vector_type="dense",
        dimension=EMBEDDINGS_DIMENSION,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        deletion_protection="disabled",
        tags={"environment": "development"},
    )

In [54]:
pc.describe_index(index_name_from_embeddings)

{
    "name": "example-index-from-embeddings",
    "metric": "cosine",
    "host": "example-index-from-embeddings-22dypz1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": {
        "environment": "development"
    }
}

### Inserción de datos

Podemos insertar datos en el índice utilizando la API de Pinecone. Se pueden insertar el texto directamente en un índice creado por un modelo o los embeddings generados por un modelo.

Primeramente obtenemos una referencia al índice. Se sugiere hacerlo mediante el `HOST`.

In [55]:
index_from_model = pc.Index(host=pc.describe_index(index_name_from_model).host)
index_from_embeddings = pc.Index(
    host=pc.describe_index(index_name_from_embeddings).host
)

#### Inserción de datos desde un modelo
<small>https://docs.pinecone.io/guides/data/upsert-data#upsert-text</small>

In [56]:
# Para utilizar un indice con un modelo incorporado, el texto tiene que estar en el mismo campo de "field_map" que el que se utilizó para crear el índice.
# En este caso, el campo es "chunk_text".
records = [
    {
        "_id": "vec1",
        "chunk_text": "AAPL reported a year-over-year revenue increase, expecting stronger Q3 demand for its flagship phones.",
        "category": "technology",
        "quarter": "Q3",
    },
    {
        "_id": "vec2",
        "chunk_text": "Analysts suggest that AAPL'''s upcoming Q4 product launch event might solidify its position in the premium smartphone market.",
        "category": "technology",
        "quarter": "Q4",
    },
    {
        "_id": "vec3",
        "chunk_text": "AAPL'''s strategic Q3 partnerships with semiconductor suppliers could mitigate component risks and stabilize iPhone production.",
        "category": "technology",
        "quarter": "Q3",
    },
    {
        "_id": "vec4",
        "chunk_text": "AAPL may consider healthcare integrations in Q4 to compete with tech rivals entering the consumer wellness space.",
        "category": "technology",
        "quarter": "Q4",
    },
]

In [57]:
# Definimos un namespace
model_namespace = "example-index-from-model-namespace"

# Insertamos los registros en el índice desde el modelo.
# Se utiliza el método upsert_records para el caso de un índice integrado con un modelo de embeddings.
index_from_model.upsert_records(namespace=model_namespace, records=records)

# Esperamos un momento para que los registros se indexen
time.sleep(10) # Wait for the upserted vectors to be indexed

#### Inserción de datos desde embeddings generados por un modelo
<small>https://docs.pinecone.io/guides/data/upsert-data#upsert-vectors</small>

Utilizamos un modelo liviano de sentence transformers para generar los embeddings. En este caso, se utiliza el modelo `all-MiniLM-L6-v2` de Hugging Face. Este modelo es un modelo de lenguaje preentrenado que se puede utilizar para generar embeddings de texto.

In [58]:
# Cargamos el modelo de embeddings
embeddings_model = SentenceTransformer(EMBEDDINGS_MODEL)

In [59]:
# Generamos los embeddings para los registros
embeddings = embeddings_model.encode([record["chunk_text"] for record in records])
print("Shape de embeddings:", embeddings.shape)
print("Ejemplo de embedding:", embeddings[0])

Shape de embeddings: (4, 384)
Ejemplo de embedding: [-9.10756961e-02  5.78680309e-03 -2.65190192e-02 -4.22077291e-02
 -5.60736023e-02  1.62679721e-02 -2.68427879e-02  2.72103734e-02
  2.61755064e-02  7.19104633e-02  2.78321765e-02  7.81621113e-02
 -5.05774356e-02  1.26873078e-02  6.76316172e-02  1.51894605e-02
  9.27492511e-03 -5.14388643e-02 -4.54725213e-02 -7.65217915e-02
  6.20329008e-02  9.40446556e-03  8.89824629e-02  2.01374888e-02
  1.27259165e-01 -2.58985404e-02 -1.28399864e-01  7.99014643e-02
  4.21899259e-02  1.03150681e-03 -2.11766432e-03  8.43720213e-02
  1.35738030e-01  3.94764217e-03  3.76884267e-02 -1.06173635e-01
 -5.77156395e-02 -2.52047498e-02 -4.83358167e-02 -3.25828828e-02
  2.16147285e-02 -1.09158205e-02 -4.67568710e-02  2.56314538e-02
 -4.81532980e-03 -1.68418810e-02 -5.12416549e-02  2.80486792e-02
  4.12212126e-03  5.84439412e-02  7.65101169e-04  2.92866975e-02
  7.70645291e-02 -2.20453590e-02 -3.34727876e-02 -5.29965349e-02
  1.17956707e-02  2.08740514e-02  2.79

El formato de los registros debe ser:

> Format your input data as records, each with the following:
> - An id field with a unique record identifier for the index namespace.
> - A values field with the dense vector values.
> - Optionally, a metadata field with key-value pairs to store additional information or context. When you query the index, you can then filter by metadata to ensure only relevant records are scanned. For more information, see Metadata Filtering.

Ejemplo:

```python
{
    "id": "unique_id",
    "values": [0.1, 0.2, 0.3, ...],
    "metadata": {
        "text": "text to be stored"
    }
}
```

In [60]:
# Creamos la estructura de datos para insertar en el índice
embeddings_data = []
for i, record in enumerate(records):
    embeddings_data.append((record["_id"], embeddings[i].tolist(), {"category": record["category"], "quarter": record["quarter"]}))
print(embeddings_data)

[('vec1', [-0.09107569605112076, 0.005786803085356951, -0.026519019156694412, -0.04220772907137871, -0.05607360228896141, 0.016267972066998482, -0.026842787861824036, 0.027210373431444168, 0.02617550641298294, 0.07191046327352524, 0.02783217653632164, 0.07816211134195328, -0.05057743564248085, 0.01268730778247118, 0.06763161718845367, 0.01518946047872305, 0.009274925105273724, -0.051438864320516586, -0.04547252133488655, -0.07652179151773453, 0.062032900750637054, 0.009404465556144714, 0.0889824628829956, 0.020137488842010498, 0.12725916504859924, -0.025898540392518044, -0.12839986383914948, 0.07990146428346634, 0.04218992590904236, 0.0010315068066120148, -0.0021176643203943968, 0.08437202125787735, 0.13573803007602692, 0.003947642166167498, 0.03768842667341232, -0.10617363452911377, -0.05771563947200775, -0.025204749777913094, -0.04833581671118736, -0.032582882791757584, 0.02161472849547863, -0.010915820486843586, -0.04675687104463577, 0.025631453841924667, -0.004815329797565937, -0.0

In [61]:
embeddings_namespace = "example-index-from-embeddings-namespace"

# Insertamos los registros en el índice desde los embeddings.
# Se utiliza el método upsert para el caso de un índice creado desde embeddings.
index_from_embeddings.upsert(vectors=embeddings_data, namespace=embeddings_namespace)

# Esperamos un momento para que los registros se indexen
time.sleep(10) # Wait for the upserted vectors to be indexed

### Búsqueda de datos

In [62]:
query_text = "What are the expectations for Apple's next flagship phone demand?"

#### Búsqueda de datos desde un modelo

In [63]:
results = index_from_model.search(
    namespace=model_namespace, 
    query={
        "inputs": {"text": query_text}, 
        "top_k": 2
    },
    fields=["category", "chunk_text"]
)

print(results)

{'result': {'hits': [{'_id': 'vec1',
                      '_score': 0.8434841632843018,
                      'fields': {'category': 'technology',
                                 'chunk_text': 'AAPL reported a year-over-year '
                                               'revenue increase, expecting '
                                               'stronger Q3 demand for its '
                                               'flagship phones.'}},
                     {'_id': 'vec2',
                      '_score': 0.8364965915679932,
                      'fields': {'category': 'technology',
                                 'chunk_text': "Analysts suggest that AAPL'''s "
                                               'upcoming Q4 product launch '
                                               'event might solidify its '
                                               'position in the premium '
                                               'smartphone market.'}}]},
 'usage': {'embed_t

#### Búsqueda de datos desde embeddings generados por un modelo

Obtenemos el embedding del texto a buscar:

In [64]:
# Genera el embedding para la consulta
query_embedding = embeddings_model.encode([query_text]).tolist()

Realizamos la consulta:

In [67]:
# Realiza la búsqueda por similitud
results = index_from_embeddings.query(
    vector=query_embedding,
    top_k=2,  # Número de resultados similares que quieres obtener
    include_values=False,  # No incluir los valores de los vectores coincidentes
    include_metadata=True,  # Incluir los metadatos de los vectores coincidentes
    namespace=embeddings_namespace,
)

print(results)

{'matches': [{'id': 'vec1',
              'metadata': {'category': 'technology', 'quarter': 'Q3'},
              'score': 0.485725492,
              'values': []},
             {'id': 'vec2',
              'metadata': {'category': 'technology', 'quarter': 'Q4'},
              'score': 0.478230476,
              'values': []}],
 'namespace': 'example-index-from-embeddings-namespace',
 'usage': {'read_units': 1}}


### Listamos los índices disponibles

In [21]:
# Listamos los índices disponibles
indexes = pc.list_indexes()
print(f"Available indexes: {indexes}")

Available indexes: [{
    "name": "example-index-from-embeddings",
    "metric": "cosine",
    "host": "example-index-from-embeddings-22dypz1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": {
        "environment": "development"
    }
}, {
    "name": "example-index-from-model",
    "metric": "cosine",
    "host": "example-index-from-model-22dypz1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null,
    "embed": {
        "model": "multilingual-

### Eliminamos los indices

Eliminamos los índices creados en el paso anterior:

In [68]:
# Eliminamos los índices creados
pc.delete_index(index_name_from_model)
pc.delete_index(index_name_from_embeddings)